<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/SigLip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Optional, Tuple
import torch
import torch.nn as nn

In [ ]:
class SiglipVisionConfig:

  def __init__(
      self,
      hidden_size=768,
      intermediate_size=3072,
      num_hidden_layers=12,
      num_attention_heads=12,
      num_channels=3,
      image_size=224,
      patch_size=16,
      layer_norm_eps=1e-6,
      attention_dropout=0.0,
      num_image_tokens: int = None,
      **kwargs
  ):
    super().__init__()

    self.hidden_size = hidden_size
    self.intermediate_size = intermediate_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.num_channels = num_channels
    self.patch_size = patch_size
    self.image_size = image_size
    self.attention_dropout = attention_dropout
    self.layer_norm_eps = layer_norm_eps
    self.num_image_tokens = num_image_tokens

In [ ]:
class SiglipVisionEmbeddings(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    self.embed_dim = config.hidden_size
    self.image_size = config.image_size
    self.patch_size = config.patch_size

    self.patch_embedding = nn.Conv2d(
        in_channels=config.num_channels,
        out_channels=self.embed_dim,
        kernel_size=self.patch_size,
        stride=self.patch_size,
        padding="valid" # this indicates that no padding is added
    )

    self.num_patches = (self.image_size // self.patch_size) ** 2
    self.num_positions = self.num_patches
    self.position_embedding = nn.Embedding(self.num_positions, self.embed_dim)
    self.register_buffer(
        "position_ids",
        torch.arange(self.num_positions).expand((1, -1)),
        persistent=False
    )

  def forward(self, pixel_values: torch.FloatTensor) -> torch.Tensor:
    _, _, height, width = pixel_values.shape # (B, C, H, W)
    patch_embeds = self.patch_embedding(pixel_values) # (B, embed_dim, num_patches[num_patches_h * num_patches_w])
    embeddings = patch_embeds.flatten(2)
    embeddings = embeddings.transpose(1, 2) # (B, num_patches, embed_dim)
    embeddings = embeddings + self.position_embedding(self.position_ids)
    return embeddings # (B, num_patches, embed_dim)


In [ ]:
class SiglipAttention(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config
    self.embed_dim = config.hidden_size
    self.num_heads = config.num_attention_heads
    self.head_dim = self.embed_dim // self.num_heads
    self.scale = self.head_dim**-0.5 # equivalent to 1 / sqrt(self.head_dim)
    self.dropout = config.attention_dropout

    self.k_proj = nn.Linear(self.embed_dim, self.embed_dim)
    self.v_proj = nn.Linear(self.embed_dim, self.embed_dim)
    self.q_proj = nn.Linear(self.embed_dim, self.embed_dim)
    self.out_proj = nn.Linear(self.embed_dim, self.embed_dim)

  def forward(
      self,
      hidden_states: torch.Tensor,
  ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
    batch_size, seq_len, _ = hidden_states.size()   # (B, num_patches, embed_dim)
    query_states = self.q_proj(hidden_states)       # (B, num_patches, embed_dim)
    key_states = self.k_proj(hidden_states)         # (B, num_patches, embed_dim)
    values_states = self.v_proj(hidden_states)      # (B, num_patches, embed_dim)
    query_states = query_states.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1,2) # (B, num_heads, num_patches, head_dim)
    key_states = key_states.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1,2)     # (B, num_heads, num_patches, head_dim)
    value_states = value_states.view(batch_size, seq_len, self.num_heads, self.head_dim)                # (B, num_heads, num_patches, head_dim)
    # attention formula: Q * K^T / sqrt(d_k)
    attn_weights = (torch.matmul(query_states, key_states.transpose(2,3)) * self.scale)                 # (B, num_heads, num_patches, num_patches)

    if attn_weights.size() != (batch_size, self.num_heads, seq_len, seq_len):
      raise ValueError(
          f"Attention weights should be of size {(batch_size, self.num_heads, seq_len, seq_len)}, but is"
          f" {attn_weights.size()}"
      )

    # Apply softmax row-wise. attn_weights (B, num_heads, num_patches, num_patches)
    attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
    # Apply dropout only during training
    attn_weights = nn.functional.dropout(attn_weights, p=self.dropout, training=self.training)
    attn_output = torch.matmul(attn_weights, values_states)

    if attn_output.size() != (batch_size, self.num_heads, seq_len, self.head_dim):
      raise ValueError(
          f"attn_output should be of size {(batch_size, self.num_heads, seq_len, self.head_dim)}, but is"
          f"{attn_output.size()}"
      )

      attn_output = attn_output.transpose(1,2).contiguous() # (B, num_patches, num_heads, head_dim)
      attn_output = attn_output.reshape(batch_size, seq_len, self.embed_dim)


In [ ]:
class SiglipMLP(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config
    self.fc1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.fc2 = nn.Linear(config.intermediate_size, config.hidden_size)

  def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
    hidden_states = self.fc1(hidden_states)
    hidden_states = nn.functional.gelu(hidden_states, approximate="tanh")
    hidden_states = self.fc2(hidden_states)
    return hidden_states

In [ ]:
class SiglipEncoderLayer(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.embed_dim = config.hidden_size
    self.self_attn = SiglipAttention(config)
    self.layer_norm1 = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_eps)
    self.mlp = SiglipMLP(config)
    self.layer_norm2 = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_eps)

  def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
    residual = hidden_states # (B, num_patches, embed_dim)
    hidden_states = self.layer_norm1(hidden_states) # (B, num_patches, embed_dim)
    hidden_states, _ = self.self_attn(hidden_states=hidden_states) # (B, num_patches, embed_dim)
    hidden_states = residual + hidden_states # (B, num_patches, embed_dim)
    residual = hidden_states
    hidden_states = self.layer_norm2(hidden_states) # (B, num_patches, embed_dim)
    hidden_states = self.mlp(hidden_states) # (B, num_patches, embed_dim)
    hidden_states = residual + hidden_states # (B, num_patches, embed_dim)

    return hidden_states


In [ ]:
class SiglipVisionTransformer(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    embed_dim = config.hidden_size
    self.embeddings = SiglipVisionEmbeddings(config)
    self.encoder = SiglipEncoder(config)
    self.post_layernorm = nn.LayerNorm(embedd_dim, eps=config.layer_norm_eps)

  def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
    # B, C, H, W -> B, num_patches, embed_dim
    hidden_states = self.embeddings(pixel_values)
    last_hidden_state = self.encoder(input_embed=hidden_states)
    last_hidden_state = self.post_layernorm(last_hidden_state)

    return last_hidden_state

class SiglipVisionModel(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    self.vision_model = SiglipVisionTransformer(config)

  def forward(self, pixel_values) -> Tuple:
    # B, C, H, W -> B, num_patches, embed_dim
    return self.vision_model(pixel_values=pixel_values)